In [1]:
from huggingface_hub import notebook_login
notebook_login()

squad_v2 = False
model_checkpoint = "distilbert-base-uncased"
batch_size = 16


In [2]:
import transformers
print(transformers.__version__)

from transformers.utils import send_example_telemetry
send_example_telemetry("question_answering_notebook", framework="pytorch")

# datasets transformers huggingface_hub ipywidgets

4.30.2


In [3]:
from datasets import load_dataset, load_metric

from datasets import load_dataset
# datasets = load_dataset("squad_v2" if squad_v2 else "squad")
datasets = load_dataset("csv", data_files="pokemon_contexts2.csv")
datasets['validation'] = datasets['train'].train_test_split(test_size=0.1, seed=42)['test']




Found cached dataset csv (C:/Users/mvinn/.cache/huggingface/datasets/csv/default-19aaf7f3202ead10/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached split indices for dataset at C:\Users\mvinn\.cache\huggingface\datasets\csv\default-19aaf7f3202ead10\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-a7d44d491cb49f6e.arrow and C:\Users\mvinn\.cache\huggingface\datasets\csv\default-19aaf7f3202ead10\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-09bf701b19f60683.arrow


In [4]:
import json

# transform the answers from string to dict
datasets['train'] = datasets['train'].map(lambda example: { **example, 'answers': json.loads(example['answers'].replace("'", '"')) })
datasets['validation'] = datasets['validation'].map(lambda example: { **example, 'answers': json.loads(example['answers'].replace("'", '"')) })

Loading cached processed dataset at C:\Users\mvinn\.cache\huggingface\datasets\csv\default-19aaf7f3202ead10\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-d5bcd61e6575e314.arrow
Loading cached processed dataset at C:\Users\mvinn\.cache\huggingface\datasets\csv\default-19aaf7f3202ead10\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-fe4d798c7d720e8d.arrow


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [6]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [7]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

for i, example in enumerate(datasets["train"]):
  if len(tokenizer(example["question"], example["context"])["input_ids"]) > max_length:
    break
example = datasets["train"][i]

In [8]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

70

In [9]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [10]:
[len(x) for x in tokenized_example["input_ids"]]

[70]

In [11]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] whats the type of missingno.? [SEP] missingno. its a bird [ nb 9 ] first generation pokemon that an error handler species, " missing number " was created to handle attempts at accessing nonexistent pokemon species. it is regarded as one of the most famous and popular glitches in video game history. [SEP]


In [12]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 4), (4, 5), (6, 9), (10, 14), (15, 17), (18, 25), (25, 27), (27, 28), (28, 29), (0, 0), (0, 7), (7, 9), (9, 10), (11, 14), (15, 16), (17, 21), (21, 22), (22, 23), (23, 24), (25, 26), (26, 27), (28, 33), (34, 44), (45, 52), (53, 57), (58, 60), (61, 66), (67, 74), (75, 82), (82, 83), (84, 85), (85, 92), (93, 99), (99, 100), (101, 104), (105, 112), (113, 115), (116, 122), (123, 131), (132, 134), (135, 141), (141, 144), (145, 149), (149, 151), (151, 155), (155, 156), (157, 164), (165, 172), (172, 173), (174, 176), (177, 179), (180, 188), (189, 191), (192, 195), (196, 198), (199, 202), (203, 207), (208, 214), (215, 218), (219, 226), (227, 228), (228, 231), (231, 235), (236, 238), (239, 244), (245, 249), (250, 257), (257, 258), (0, 0)]


In [13]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

what What


In [14]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


In [15]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

16 21


In [16]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

bird [ nb 9 ]
Bird[nb 9]


In [17]:
pad_on_right = tokenizer.padding_side == "right"

In [18]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [19]:
features = prepare_train_features(datasets['train'][:5])

In [20]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)


Loading cached processed dataset at C:\Users\mvinn\.cache\huggingface\datasets\csv\default-19aaf7f3202ead10\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-370335fac21d5ae0.arrow


Loading cached processed dataset at C:\Users\mvinn\.cache\huggingface\datasets\csv\default-19aaf7f3202ead10\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-34432ef397cc1928.arrow


In [21]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to

In [22]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [23]:
from transformers import default_data_collator

data_collator = default_data_collator

In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()
trainer.save_model("pokemon-gpt-trained")

Cloning https://huggingface.co/M4ycon/distilbert-base-uncased-finetuned-squad into local empty directory.
c:\Python311\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 4.363717555999756, 'eval_runtime': 6.0361, 'eval_samples_per_second': 2.651, 'eval_steps_per_second': 0.166, 'epoch': 1.0}


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 2.4965271949768066, 'eval_runtime': 5.6892, 'eval_samples_per_second': 2.812, 'eval_steps_per_second': 0.176, 'epoch': 3.0}
{'train_runtime': 470.8884, 'train_samples_per_second': 0.968, 'train_steps_per_second': 0.064, 'train_loss': 4.041168721516927, 'epoch': 3.0}


Upload file pytorch_model.bin:   0%|          | 1.00/253M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.87k [00:00<?, ?B/s]

To https://huggingface.co/M4ycon/distilbert-base-uncased-finetuned-squad
   a26bd9e..678c0d5  main -> main

To https://huggingface.co/M4ycon/distilbert-base-uncased-finetuned-squad
   678c0d5..3b5c59b  main -> main

